<a href="https://colab.research.google.com/github/viett887g/2023-10-27-baitapML/blob/main/redo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
import numpy as np
import os
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd

# **I. Chọn dataset :USA Housing**

In [28]:
def readdata(file):
    data = pd.read_csv(file)
    del data['Address']
    data = np.array(data)
    return data

# Hàm chuẩn hóa dữ liệu và tạo training set và test set
def create_data(data):
    data = preprocessing.MinMaxScaler().fit_transform(data)
    X = data[:,:-1]
    y = data[:, -1]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    return X_train, X_test, y_train, y_test

# Hàm sigmoid
def sigmoid(X, w):
    Hw = 1 / (1 + np.exp(-np.dot(X, w)))
    return Hw

def linear_loss(X, y, w):
      m = X.shape[0]
      y_pred = np.dot(X, w)
      J = 1/(2*m) * np.sum((y_pred - y)**2)
      return J

# II. Chọn 2 phương pháp **Gradient Descent (GD)**, **Momentum Based GD** tối ưu để tối ưu hoá loss function Linear Regression cho tập dữ liệu USA Housing

##**2.1 Gradient Descent (GD)**

In [29]:
def gradient_descent(X, y, w, learning_rate, iterations):
    m = X.shape[0]
    for i in range(iterations):
        h = np.dot(X, w)
        loss = h - y
        gradient = np.dot(X.T, loss) / m
        w -= learning_rate * gradient
    return w

## **2.2 Momentum Based GD**

In [30]:
def momentum_gradient_descent(X, y, w, learning_rate, gamma, iterations):
    m = X.shape[0]
    v = np.zeros_like(w)  # Khởi tạo véc-tơ momentum ban đầu là 0

    for i in range(iterations):
        h = np.dot(X, w)
        loss = h - y
        gradient = np.dot(X.T, loss) / m

        v = gamma * v + learning_rate * gradient  # Cập nhật momentum theo gradient hiện tại và momentum trước đó
        w -= v  # Cập nhật trọng số dựa trên momentum

    return w


# **III. So sánh hiệu suất của các thuật toán**

In [31]:
def evaluate_performance(X_test, y_test, w):
    # Tính toán dự đoán trên tập test
    y_pred = np.dot(X_test, w)

    # Tính toán Mean Squared Error (MSE)
    mse = mean_squared_error(y_test, y_pred)

    # Tính toán Rmse
    rmse = np.sqrt(mse)


    # Trả về kết quả hiệu suất của mô hình
    return mse, rmse

In [42]:
def main():
    file = ("/content/drive/MyDrive/USA_Housing.csv")  # Thay đổi thành tên tệp dữ liệu thực tế của bạn
    data = readdata(file)
    X_train, X_test, y_train, y_test = create_data(data)

    num_features = X_train.shape[1]
    w = np.zeros(num_features)  # Khởi tạo trọng số ban đầu

    learning_rate = 0.01
    num_iterations = 1000
    gamma = 0.9

    # Sử dụng Gradient Descent
    w_optimal_gd = gradient_descent(X_train, y_train, w, learning_rate, num_iterations)
    loss_gd = linear_loss(X_train, y_train, w_optimal_gd)

    print("Giá trị loss tối ưu của Gradient Descents :", loss_gd)
    mse, rmse = evaluate_performance(X_test, y_test, w_optimal_gd)

    print("Mean Squared Error của Gradient Descents (MSE):", mse)
    print('Root Mean Squared Error của Gradient Descents:',rmse)
    print('\n','\n')

    # Sử dụng Momentum Based Gradient Descent
    w_optimal_momentum = momentum_gradient_descent(X_train, y_train, w, learning_rate, gamma, num_iterations)
    loss_momentum = linear_loss(X_train, y_train, w_optimal_momentum)
    print("Giá trị loss tối ưu của momentum_based_gradient_descent :", loss_momentum)
    mse, rmse = evaluate_performance(X_test, y_test, w_optimal_momentum)

    print("Mean Squared Error (MSE) của momentum_based_gradient_descent:", mse)
    print('Root Mean Squared Error của momentum_based_gradient_descent:',rmse)


if __name__ == "__main__":
    main()



Giá trị loss tối ưu của Gradient Descents : 0.004147692715904917
Mean Squared Error của Gradient Descents (MSE): 0.008215352544588292
Root Mean Squared Error của Gradient Descents: 0.09063858198685751

 

Giá trị loss tối ưu của momentum_based_gradient_descent : 0.0035506389587542883
Mean Squared Error (MSE) của momentum_based_gradient_descent: 0.0069475994237732435
Root Mean Squared Error của momentum_based_gradient_descent: 0.08335226105975316


# **IV. Nhận xét**

mô hình momentum_based_gradient_descent có giá trị loss tối ưu thấp hơn, cho thấy mô hình này có hiệu suất tốt hơn trong việc tối ưu hóa hàm mất mát.

mô hình momentum_based_gradient_descent có giá trị MSE thấp hơn, cho thấy mô hình này có khả năng dự đoán chính xác hơn và đạt được mức độ biến động giữa giá trị dự đoán và giá trị thực tế tốt hơn.

mô hình momentum_based_gradient_descent có giá trị RMSE thấp hơn, cho thấy mô hình này có khả năng dự đoán chính xác hơn và có độ lệch trung bình giữa giá trị dự đoán và giá trị thực tế thấp hơn.

# **IV. Kết luận**
Dựa trên các kết quả trên, mô hình momentum_based_gradient_descent có giá trị loss thấp hơn và các chỉ số MSE và RMSE nhỏ hơn, do đó có thể xem là mô hình này tối ưu hơn và tốt hơn so với mô hình Gradient Descents.

# **V. Ưu nhược điểm của phương pháp tối ưu.**

**Ưu điểm của phương pháp tối ưu Gradient Descent (GD):**

Đơn giản và dễ hiểu: Gradient Descent là một phương pháp tối ưu đơn giản và dễ hiểu. Nó chỉ yêu cầu tính đạo hàm đơn giản của hàm mất mát, và cập nhật các tham số theo hướng giảm độ dốc của đạo hàm.

Hiệu quả tính toán: GD thường được sử dụng trong nhiều mô hình học máy công nghệ cao, bởi vì nó có thể xử lý tập dữ liệu lớn một cách hiệu quả. Việc tính toán gradient chỉ phụ thuộc vào kích thước của batch, và không yêu cầu tính toán ma trận đầy đủ.

Hội tụ tới điểm tối ưu cục bộ: Nếu được cài đặt đúng, GD có khả năng hội tụ tới điểm tối ưu cục bộ. Nếu hàm mất mát là lồi và tốt hơn, GD có thể cung cấp giải pháp tối ưu đối tượng.

**Nhược điểm của phương pháp tối ưu Gradient Descent (GD):**

Tốc độ hội tụ chậm: GD có thể hội tụ chậm đối với một số bài toán, đặc biệt là khi hàm mất mát có đặc trưng vùng trong lỏng lẻo hoặc bề mặt lõm. Việc cập nhật các tham số trên toàn bộ dữ liệu có thể cần nhiều vòng lặp để đạt điểm tối ưu.
Momentum Based Gradient Descent (GD với đà):


**Ưu điểm Momentum Based GD** có thể giúp cải thiện tốc độ hội tụ của GD. Bằng cách tích lũy đà từ các bước trước, nó có khả năng vượt qua các mục tiêu hẹp và giảm thiểu trỗi dạng cục bộ, đồng thời giảm ảnh hưởng của các điểm nhiễu trên đường đi hội tụ.

**Nhược điểm Momentum Based GD**: Mặc dù hiệu quả hơn so với GD cơ bản, tuy nhiên không phải lúc nào Momentum Based GD cũng tốt hơn. Trong một số trường hợp, điều này có thể gây ra quá đà, kéo dài thời gian hội tụ hoặc không hội tụ tới điểm tối ưu. Bên cạnh đó, việc điều chỉnh thông số đà có thể không dễ dàng và yêu cầu thử nghiệm.